In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam


In [2]:
# Membaca file CSV
df = pd.read_csv('data_harga_beras.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date').interpolate(method='linear').reset_index()

In [3]:
# Drop kolom yang tidak digunakan
df = df.drop(['C4Medium','Bulog'], axis=1)
df = df.filter(['C4Super'])
df = df.values.astype('float32')

In [4]:
# Normalisasi data dengan range [0, 1] sebelum membagi data
scaler_0_1 = MinMaxScaler(feature_range=(0, 1))
df_normalized_0_1 = scaler_0_1.fit_transform(df)

scaler_neg1_1 = MinMaxScaler(feature_range=(-1, 1))
df_normalized_neg1_1 = scaler_neg1_1.fit_transform(df)

In [5]:
# Fungsi untuk menyiapkan data
def prepare_data(df, timesteps):
    X, Y = [], []
    for i in range(len(df) - timesteps - 1):
        X.append(df[i:(i + timesteps)])
        Y.append(df[i + timesteps, 0])
    return np.array(X), np.array(Y)

In [6]:
# Fungsi untuk membangun dan melatih model
def build_and_train_model(df_normalized, timesteps=1, num_neuron=10, num_epoch=50, num_batch=16):
    # Split data menjadi training dan testing
    n_samples = len(df_normalized)
    train_size = int(n_samples * 0.8)
    test_size = n_samples - train_size

    train_data = df_normalized[:train_size]
    test_data = df_normalized[train_size:]
    
    # Persiapan data
    X_train, Y_train = prepare_data(train_data, timesteps)
    X_test, Y_test = prepare_data(test_data, timesteps)

    # Reshape data
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    # Build model
    model = Sequential()
    model.add(LSTM(4, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(num_neuron, activation='tanh', recurrent_activation='sigmoid', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, bias_initializer='zeros'))
    
    # Compile model
    adam_optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=adam_optimizer, loss='mse', metrics=['MAPE'])
    
    # Train model
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=num_epoch, batch_size=num_batch, verbose=1)
    
    return model, X_test, Y_test, history


In [7]:
# Normalisasi dengan range [0, 1]
model_0_1, X_test_0_1, Y_test_0_1, history_0_1 = build_and_train_model(df_normalized_0_1)

# Normalisasi dengan range [-1, 1]
model_neg1_1, X_test_neg1_1, Y_test_neg1_1, history_neg1_1 = build_and_train_model(df_normalized_neg1_1)

Epoch 1/50


C:\Users\Windows\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


61/61 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - MAPE: 86776.8359 - loss: 0.0244 - val_MAPE: 79.9773 - val_loss: 0.3662
Epoch 2/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 133367.3438 - loss: 0.0130 - val_MAPE: 72.9479 - val_loss: 0.3074
Epoch 3/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - MAPE: 537626.8125 - loss: 0.0126 - val_MAPE: 68.6956 - val_loss: 0.2720
Epoch 4/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 610155.6875 - loss: 0.0109 - val_MAPE: 61.2297 - val_loss: 0.2145
Epoch 5/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - MAPE: 436824.2500 - loss: 0.0082 - val_MAPE: 43.6617 - val_loss: 0.1085
Epoch 6/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - MAPE: 218370.7344 - loss: 0.0054 - val_MAPE: 25.6126 - val_loss: 0.0358
Epoch 7/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 168281.2344 - loss: 0.0027 - val_MAPE: 6.0769 - val_loss: 0.0020
Epoch 8/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - MAPE: 199024.9531 - loss: 0.0022 - val_MAPE: 3.4322 - val_loss: 0.0012
Epoch 9/50
61/61 ━━━━

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 65.5602 - loss: 0.3107 - val_MAPE: 203.5983 - val_loss: 0.6078
Epoch 3/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 45.2359 - loss: 0.1097 - val_MAPE: 256.0566 - val_loss: 0.7968
Epoch 4/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 43.9262 - loss: 0.0523 - val_MAPE: 259.3868 - val_loss: 0.8034
Epoch 5/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - MAPE: 45.8827 - loss: 0.0504 - val_MAPE: 251.1956 - val_loss: 0.7660
Epoch 6/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 40.8793 - loss: 0.0466 - val_MAPE: 247.0536 - val_loss: 0.7431
Epoch 7/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 49.8393 - loss: 0.0476 - val_MAPE: 239.9539 - val_loss: 0.7100
Epoch 8/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 36.7195 - loss: 0.0421 - val_MAPE: 230.8478 - val_loss: 0.6712
Epoch 9/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - MAPE: 33.1130 - loss: 0.0388 - val_MAPE: 221.3603 - val_loss: 0.6329
Epoch 10/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4

In [8]:
# Evaluasi model sebelum denormalisasi
def evaluate_model_before_denormalization(model, X_test, Y_test):
    # Predict
    test_predict = model.predict(X_test, verbose=0)

    # Calculate metrics
    mae = mean_absolute_error(Y_test, test_predict)
    rmse = sqrt(mean_squared_error(Y_test, test_predict))
    mape = mean_absolute_percentage_error(Y_test, test_predict) * 100

    return mae, rmse, mape, Y_test, test_predict

In [9]:

# Evaluasi model untuk kedua skala
mae_0_1, rmse_0_1, mape_0_1, Y_test_0_1, test_predict_0_1 = evaluate_model_before_denormalization(model_0_1, X_test_0_1, Y_test_0_1)
mae_neg1_1, rmse_neg1_1, mape_neg1_1, Y_test_neg1_1, test_predict_neg1_1 = evaluate_model_before_denormalization(model_neg1_1, X_test_neg1_1, Y_test_neg1_1)

# Print hasil evaluasi
print(f"Interval [0, 1] -> RMSE: {rmse_0_1}, MAPE: {mape_0_1}%")
print(f"Interval [-1, 1] -> RMSE: {rmse_neg1_1}, MAPE: {mape_neg1_1}%")


Interval [0, 1] -> RMSE: 0.04376130655125724, MAPE: 4.930485785007477%
Interval [-1, 1] -> RMSE: 0.5661027818844274, MAPE: 129.57017421722412%
